### Install deps

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%pip install uv dotenv
!uv pip install datasets transformers sentencepiece peft accelerate bitsandbytes trl xformers nltk rouge-score evaluate
!uv pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Using Python 3.11.11 environment at: /usr
Audited 11 packages in 149ms
Using Python 3.11.11 environment at: /usr
Resolved 76 packages in 905ms
Audited 76 packages in 0.20ms


In [5]:
import dotenv
import os

DRIVE = "/content/drive/MyDrive/"
# DRIVE = "/content"
os.environ["WANDB_DISABLED"] = "true"
COLUMNS = ["title", "description"]

# dotenv.load_dotenv(f"{DRIVE}/.env", override=True, verbose=True)
dotenv.load_dotenv("/Users/carlosrian/Downloads/tech-challenge-03/.env", override=True, verbose=True)

True

In [10]:
from huggingface_hub import login

login(token=os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


### Read Data Using Dataset

In [8]:
# Read Data Using Dataset lib
from datasets import load_dataset

raw_dataset = load_dataset('json', data_files={'train': f'{DRIVE}/clean_dataset.json'}, split='train')

raw_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title', 'description'],
    num_rows: 2749
})

In [9]:
!nvidia-smi

Sun Mar 23 16:19:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Tokenize Datasets

In [10]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TrainingArguments
from trl import SFTTrainer

# model_checkpoint = "unsloth/llama-3-8b-bnb-4bit"
# model_checkpoint = "TheBloke/Llama-2-7B-Chat-GPTQ"  # Já vem otimizado
# max_seq_length = 2048
model_checkpoint = "mistralai/Mistral-7B-v0.1"
max_seq_length = 1024
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_checkpoint,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

==((====))==  Unsloth 2025.3.18: Fast Mistral patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Unsloth 2025.3.18 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:

prompt = """
Below is an instruction that describes a product, Based on 'Input' you must provide the Output.

### Instruction:
You must use the 'Output' to answer the 'Input' question!

### Input:
{input}

### Output:
{output}
"""

EOS_TOKEN = tokenizer.eos_token

def preprocess_function(examples: dict):
    texts = [prompt.format(input=title, output=description) + EOS_TOKEN
            for title, description in zip(examples["title"], examples["description"])]
    return { "text" : texts}

num_proc = os.cpu_count()
print(f"Number of processors: {num_proc}")
tokenized_datasets = raw_dataset.map(preprocess_function, batched=True, num_proc=num_proc)

# save tokenized dataset
tokenized_datasets.save_to_disk(f"{DRIVE}/tokenized_datasets")

# remove columns unused
tokenized_datasets = tokenized_datasets.remove_columns(["title", "description"])
tokenized_datasets

Number of processors: 2


Map (num_proc=2):   0%|          | 0/2749 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2749 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 2749
})

### Train fine tuning model

In [12]:
output_dir = f"{model_checkpoint}-finetuned-amazon-reviews-2.8k"
max_steps = 1000  # Try at least 1000+ for meaningful training
learning_rate = 1e-4  # Lower LR slightly for better convergence


training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,  # ✅ Adjust batch size as needed
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,  # ✅ Increased for stability
    num_train_epochs=3,
    max_steps=max_steps,  # ✅ Increased (20 was too low!)
    learning_rate=learning_rate,  # ✅ Lowered for better fine-tuning
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=100,
    # optim="adamw_8bit",
    optim="adamw_torch",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_datasets,
    args = training_args
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2749 [00:00<?, ? examples/s]

In [13]:
# trainer the model
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'
trainer_stats = trainer.train()
# trainer.train()
trainer.push_to_hub()
trainer_stats

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,749 | Num Epochs = 3 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
100,1.574500
200,1.480300
300,1.467500
400,1.307000
500,1.250200
600,1.257300
700,1.167100
800,1.015900
900,1.036600
1000,0.988400


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1742746919.375c0aafdb9c.2315.0:   0%|          | 0.00/8.46k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

TrainOutput(global_step=1000, training_loss=1.2544789581298827, metrics={'train_runtime': 7601.5389, 'train_samples_per_second': 1.052, 'train_steps_per_second': 0.132, 'total_flos': 1.2907159893558067e+17, 'train_loss': 1.2544789581298827})

### Tests

In [12]:
prompt_template = """
Below is an instruction that describes a product, Based on Input you must provide the Response.

### Instruction:
Summarize the product title

### Input:
{input}

### Response:
"""

from transformers import pipeline
generator = pipeline("text-generation", model="carlosrian/Mistral-7B-v0.1-finetuned-amazon-reviews-2.8k", device="cuda")

ValueError: Unrecognized model in carlosrian/Mistral-7B-v0.1-finetuned-amazon-reviews-2.8k. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, depth_pro, deta, detr, diffllama, dinat, dinov2, dinov2_with_registers, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, emu

In [ ]:

title = "The Gift Wrap Company Wrapping Paper, 37.5 Square Feet Flat Wrap, Cerise Krafty Color 100% Recycled Wrap"
question = prompt_template.format(input=title)
output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
print(output["generated_text"].split("."))


# Founded in 1903, The Gift Wrap Company is the oldest established gift wrap manufacturer in the USA.
# Over the past 100+ years, The Gift Wrap Company has grown and expanded to include modern artists and classic styles.
# Our gift wrap is 50lb to 60lb weight, the thick paper that you can keep to re-use and re-wrap the next present or even line your drawers.
# Look for coordinating gift bags, ribbons, stationery, and tags. Once you've shopped and selected the gift, be sure to wrap it with equal care.
# When selecting our holiday wrap you can rest assured that the little ones in your house won't be able to peek through!
# Keep on hand for parties; never rush to the store again."


In [ ]:
title = "White Sierra Women's Sierra Point Convertible Pant (29-Inch Inseam)"
question = prompt_template.format(input=title)
output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
print(output["generated_text"].split("."))


# From mountain trails to sunny river banks to scorching deserts,
# the Sierra Point Convertible Pant swiftly converts from pants to shorts when the temps change.
# The lightweight nylon Sierra Cloth woven fabric dries just as quickly.

In [ ]:
title = "Zaggora Women's Hot Top"
question = prompt_template.format(input=title)
output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
print(output["generated_text"].split("."))


# Zaggora Hot Tops - Small-Black - Hot Tops gets you hot in style - CELU-LITE Technology The Zaggora Hot Line of
# products has been extensively tested in both the US and UK for effectiveness.